Importing libraries

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

Importing the wiki table as a dataframe and deleting rows that have the Borough not assigned:

In [ ]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df=df[~df.Borough.str.contains("Not assigned")]
df.reset_index(drop=True, inplace=True)
df

Checking how many cells have not assigned neighbourhood:

In [ ]:
df.loc[df['Neighbourhood'] == 'Not assigned']

Since there is only one cell who has the neighbourhood not assigned I can change only that cell:

In [ ]:
df.at[5,'Neighbourhood']="Queen's Park"

Grouping rows that have the same Neighbourhood:

In [ ]:
df = df.groupby('Postcode').agg({'Borough':'first','Neighbourhood': ', '.join}).reset_index()
df

In [ ]:
df.shape